#8-Puzzle Programming - A Star

In [1]:
import heapq as hq # Heap used to represent queue
global_steps=0
global_steps2=0
from queue import LifoQueue
import copy
from time import perf_counter


def calc_manhattan_dist(state, goal):
    all_dist = []

    for i in range(0, len(state)):
        for j in range(0, len(state[i])):
            if (state[i][j] == goal[i][j]):
                continue
                
            if (state[i][j] == 0):
                continue
            
            else:
                # find where this tile is in the goal state
                i_of_goal_tile, j_of_goal_tile = find_in_sublists(state[i][j], goal)
                
                distance = abs(i - i_of_goal_tile) + abs(j - j_of_goal_tile)
                all_dist.append(distance)
    
    return sum(all_dist)


def calc_misplaced_tiles(state, goal):
    """Calculate the number of misplaced tiles. 0 is ignored in this calculation."""
    # Flatten the list of lists for easy comparison
    state = [item for sublist in state for item in sublist]
    goal = [item for sublist in goal for item in sublist]
    
    count = 0
    
    for i in range(0, len(goal)):
        if state[i] == 0:
            continue
        if (state[i] != goal[i]):
            count += 1
            
    return count


def check_states_equal(state_a, state_b):
    # Checks if two states are equal
    # Flatten the list of lists for easy comparison
    flat_list_a = [item for sublist in state_a for item in sublist]
    flat_list_b = [item for sublist in state_b for item in sublist]
    
    return flat_list_a == flat_list_b


def find_in_sublists(val, lst):
    """
    Helper function used in Node.expand. Find value in list of lists.
     
    Returns:
    A tuple containing the indices of val in lst.
    """
    for i, sub_list in enumerate(lst):
        try:
            j = sub_list.index(val)
        except ValueError:
            continue
        return i, j
            
    return None, None



class Node:
    def __init__(self, parent=None, state=0, cost_from_start=0, cost_to_goal=0):
        self.parent = parent
        self.state = state
        self.cost_from_start = cost_from_start
        self.cost_to_goal = cost_to_goal
        self.children = []
  
    # This is how we compare the states of two nodes in heapq and the explored list
    def __eq__(self, other):
        return self.state == other.state
    
    def __lt__(self, other):
        """This will automatically calculate the f-val = g(n) + h(n) when this node is put in a heapq"""
        return (self.cost_from_start + self.cost_to_goal) < (other.cost_from_start + other.cost_to_goal)

    def add_child(self, node, cost_to_expand=1):
        """Adds a child node to the list of this node's children"""
        node.cost_from_start = self.cost_from_start + cost_to_expand # extend the child'd cost from start
        node.parent = self # set parent
        self.children.append(node)
        
    def get_f_val(self):
        """Returns cost from start + cost to goal"""
        return (self.cost_from_start + self.cost_to_goal)
    
    def traceback(self):
        """Traverse all parents and print their states and traversed count"""
        self.print_state()
        stack = LifoQueue(maxsize=40)
        p = self.parent
        count = 1
        
        while p:
            count += 1
            #print("  From ","\n" )
            #p.print_state()
            stack.put(p)       
            p = p.parent
            
        print(str(count) + " nodes traced.\n")
        
        while (not stack.empty()):
            p2=stack.get()
            p2.print_state()
            print("  to ","\n" )
            
            
    def print_state(self):
        for i in self.state:
            print(i)
        
    def expand(self):
        """Returns a list of valid expanded states."""
        # find index of 0
        i_zero, j_zero = find_in_sublists(0, self.state)
        state_len = len(self.state)
        
        states_to_return = []
        
        # Move Up: Swap zero and the element above it such element exists
        if (i_zero != 0):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero - 1][j_zero]
            tmp_state[i_zero - 1][j_zero] = 0
            
            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None) 
            else:
                states_to_return.append(tmp_state)
        
        # Move Right: Swap zero and the element on its right if possible
        if (j_zero != (state_len - 1)):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero][j_zero + 1]
            tmp_state[i_zero][j_zero + 1] = 0
            
            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None) 
            else:
                states_to_return.append(tmp_state)
                
        # Move Down: Swap zero and the element below it if possible
        if (i_zero != (state_len - 1)):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero + 1][j_zero]
            tmp_state[i_zero + 1][j_zero] = 0
            
            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None) 
            else:
                states_to_return.append(tmp_state)
        
        # Move Left: Swap zero and the element on its left if possible
        if (j_zero != 0):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero][j_zero - 1]
            tmp_state[i_zero][j_zero - 1] = 0
            
            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None)
            else:
                states_to_return.append(tmp_state)
               
       
        return states_to_return

def tree_search(start_state, goal_state, heuristic, verbose="y"):
    root = Node(state=start_state)
    h = [] # Used as a heapq
    hq.heappush(h, root)
    explored = []
    
    max_nodes = 1
    expanded = 0
    
    # While heapq is not empty...
    while h:
        max_nodes = max(len(h), max_nodes)
        
        current = hq.heappop(h)
        
        #if (verbose == "y"):
            #print("\nThe best state to expand with g(n) = "
            #      + str(int(current.cost_from_start))
            #      + " and h(n) = " 
             #     + str(int(current.cost_to_goal))
            #     + " is...")
            #print("Loading ")
            #current.print_state()
        
        if (check_states_equal(current.state, goal_state)):
            if (verbose == "y"):
                print("\nFound a solution!")
                print("Traceback:")
                
                current.traceback()
                print(" " ,"\n")
                current.print_state()
                print("To solve this problem the search algorithm expanded a total of " + str(expanded) + " nodes.") 
                print("The maximum number of nodes in the queue at any one time: " + str(max_nodes))
            
            return expanded, max_nodes
        
        else:
            explored.append(current)
            
            # Filter out None states
            expanded_states_list = [non_none_state for non_none_state in current.expand() if non_none_state]
            
            # If expanded_states_list is empty after we filter all None, continue
            if expanded_states_list == []:
                continue
        
            for expanded_state in expanded_states_list:
                new_node = Node(state=expanded_state)

                # If h is not empty and new_node already exists in h, continue
                # If explored is not empty and new_node already exists in explored, continue
                if ((h and new_node in h) or (explored and new_node in explored)):
                    # already seen
                    continue

                # Depending on the heuristic, calculate costs
                if (heuristic == "a_star_misplaced"):
                    new_node.cost_to_goal = calc_misplaced_tiles(new_node.state, goal_state)
                    

                if (heuristic == "a_star_manhattan"):
                    new_node.cost_to_goal = calc_manhattan_dist(new_node.state, goal_state)
                    #print(new_node.cost_to_goal)

                # if (heuristic == "XYZ") ADD MORE HEURISTIC CHOICES HERE
                #current.print_state()
                #print("To ", "\n")
                current.add_child(node=new_node)

                
                # NOTE: F-VAL = (g(n) + h(n)) IS AUTOMATICALLY CALCULATED WHEN HEAPQ SORTS NODES
                # SEE def __lt__(self, other) IN THE NODE CLASS
                hq.heappush(h, new_node)
        
            expanded += 1
        
    print("Couldn't find a solution :(")            
    return -1






In [2]:
def print_result():
  
    avg1=0
    avg2=0   
    goal_state = [[0,1,2],[3,4,5],[6,7,8]]
    count=0
    with open('Input8PuzzleCases.txt', 'r') as f:
    # Read the file contents
        file_contents = f.read()

# Split the contents into individual lines
        lines = file_contents.strip().split('\n')

    # Process each line to create an initial state
    for line in lines:
        # Convert the line into a list of integers
        numbers = [int(num) for num in line.strip().split(',')]

        # Create the initial state
        init_state = []
        for i in range(0, len(numbers), 3):
            row = numbers[i:i+3]
            init_state.append(row)

        # Print the initial state for verification
        count=count+1
        print("############################### A Star Manhattan ###############################################")
        print(init_state)
        
        verbose = "y"
        alg = "a_star_manhattan"
        t1_start = perf_counter()
        tree_search(init_state, goal_state, alg, verbose)
        t1_stop = perf_counter()
        ms = (t1_stop - t1_start)
        avg1=ms+avg1
        print("--- %s seconds for manhattan ---" % ms)
        
        ########## Misplaced Tiles
        print("############################### A Star Misplaced ###############################################")
        print(init_state)
        verbose2 = "y"
        alg2 = "a_star_misplaced"
        t1_start1 = perf_counter()
        tree_search(init_state, goal_state, alg2, verbose2)
        t1_stop2 = perf_counter()
        ms2 = (t1_stop2 - t1_start1)
        print("--- %s seconds For misplaced ---" % ms2)
        avg2=ms2+avg2
        
  
    print("\n")
    print("average time for astar Manhattan is ",avg1/count)
    print("\n")
    print("average time for astar Misplaced is ",avg2/count)
    
  
        
    return

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
## ===============================

In [3]:
#you need to implement print_result function to print out the result according to the required format
print_result()

############################### A Star Manhattan ###############################################
[[8, 7, 5], [4, 1, 2], [3, 0, 6]]

Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
26 nodes traced.

[8, 7, 5]
[4, 1, 2]
[3, 0, 6]
  to  

[8, 7, 5]
[4, 0, 2]
[3, 1, 6]
  to  

[8, 7, 5]
[4, 2, 0]
[3, 1, 6]
  to  

[8, 7, 0]
[4, 2, 5]
[3, 1, 6]
  to  

[8, 0, 7]
[4, 2, 5]
[3, 1, 6]
  to  

[8, 2, 7]
[4, 0, 5]
[3, 1, 6]
  to  

[8, 2, 7]
[4, 1, 5]
[3, 0, 6]
  to  

[8, 2, 7]
[4, 1, 5]
[3, 6, 0]
  to  

[8, 2, 7]
[4, 1, 0]
[3, 6, 5]
  to  

[8, 2, 0]
[4, 1, 7]
[3, 6, 5]
  to  

[8, 0, 2]
[4, 1, 7]
[3, 6, 5]
  to  

[8, 1, 2]
[4, 0, 7]
[3, 6, 5]
  to  

[8, 1, 2]
[0, 4, 7]
[3, 6, 5]
  to  

[0, 1, 2]
[8, 4, 7]
[3, 6, 5]
  to  

[1, 0, 2]
[8, 4, 7]
[3, 6, 5]
  to  

[1, 4, 2]
[8, 0, 7]
[3, 6, 5]
  to  

[1, 4, 2]
[0, 8, 7]
[3, 6, 5]
  to  

[1, 4, 2]
[3, 8, 7]
[0, 6, 5]
  to  

[1, 4, 2]
[3, 8, 7]
[6, 0, 5]
  to  

[1, 4, 2]
[3, 0, 7]
[6, 8, 5]
  to  

[1, 4, 2]
[3, 7, 0]
[6, 8, 5]
 


Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
26 nodes traced.

[3, 8, 4]
[0, 1, 7]
[5, 2, 6]
  to  

[3, 8, 4]
[5, 1, 7]
[0, 2, 6]
  to  

[3, 8, 4]
[5, 1, 7]
[2, 0, 6]
  to  

[3, 8, 4]
[5, 1, 7]
[2, 6, 0]
  to  

[3, 8, 4]
[5, 1, 0]
[2, 6, 7]
  to  

[3, 8, 0]
[5, 1, 4]
[2, 6, 7]
  to  

[3, 0, 8]
[5, 1, 4]
[2, 6, 7]
  to  

[3, 1, 8]
[5, 0, 4]
[2, 6, 7]
  to  

[3, 1, 8]
[0, 5, 4]
[2, 6, 7]
  to  

[3, 1, 8]
[2, 5, 4]
[0, 6, 7]
  to  

[3, 1, 8]
[2, 5, 4]
[6, 0, 7]
  to  

[3, 1, 8]
[2, 0, 4]
[6, 5, 7]
  to  

[3, 1, 8]
[0, 2, 4]
[6, 5, 7]
  to  

[0, 1, 8]
[3, 2, 4]
[6, 5, 7]
  to  

[1, 0, 8]
[3, 2, 4]
[6, 5, 7]
  to  

[1, 2, 8]
[3, 0, 4]
[6, 5, 7]
  to  

[1, 2, 8]
[3, 4, 0]
[6, 5, 7]
  to  

[1, 2, 0]
[3, 4, 8]
[6, 5, 7]
  to  

[1, 0, 2]
[3, 4, 8]
[6, 5, 7]
  to  

[1, 4, 2]
[3, 0, 8]
[6, 5, 7]
  to  

[1, 4, 2]
[3, 5, 8]
[6, 0, 7]
  to  

[1, 4, 2]
[3, 5, 8]
[6, 7, 0]
  to  

[1, 4, 2]
[3, 5, 0]
[6, 7, 8]
  to  

[1, 4, 2]
[3, 0, 5]
[6, 7, 8]
  to  

[1, 0, 2]



Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
29 nodes traced.

[6, 8, 5]
[3, 4, 7]
[2, 1, 0]
  to  

[6, 8, 5]
[3, 4, 7]
[2, 0, 1]
  to  

[6, 8, 5]
[3, 0, 7]
[2, 4, 1]
  to  

[6, 8, 5]
[0, 3, 7]
[2, 4, 1]
  to  

[0, 8, 5]
[6, 3, 7]
[2, 4, 1]
  to  

[8, 0, 5]
[6, 3, 7]
[2, 4, 1]
  to  

[8, 3, 5]
[6, 0, 7]
[2, 4, 1]
  to  

[8, 3, 5]
[0, 6, 7]
[2, 4, 1]
  to  

[8, 3, 5]
[2, 6, 7]
[0, 4, 1]
  to  

[8, 3, 5]
[2, 6, 7]
[4, 0, 1]
  to  

[8, 3, 5]
[2, 0, 7]
[4, 6, 1]
  to  

[8, 3, 5]
[0, 2, 7]
[4, 6, 1]
  to  

[0, 3, 5]
[8, 2, 7]
[4, 6, 1]
  to  

[3, 0, 5]
[8, 2, 7]
[4, 6, 1]
  to  

[3, 2, 5]
[8, 0, 7]
[4, 6, 1]
  to  

[3, 2, 5]
[0, 8, 7]
[4, 6, 1]
  to  

[3, 2, 5]
[4, 8, 7]
[0, 6, 1]
  to  

[3, 2, 5]
[4, 8, 7]
[6, 0, 1]
  to  

[3, 2, 5]
[4, 0, 7]
[6, 8, 1]
  to  

[3, 2, 5]
[4, 7, 0]
[6, 8, 1]
  to  

[3, 2, 5]
[4, 7, 1]
[6, 8, 0]
  to  

[3, 2, 5]
[4, 7, 1]
[6, 0, 8]
  to  

[3, 2, 5]
[4, 0, 1]
[6, 7, 8]
  to  

[3, 2, 5]
[4, 1, 0]
[6, 7, 8]
  to  

[3, 2, 0]



Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
25 nodes traced.

[8, 5, 1]
[7, 3, 2]
[4, 6, 0]
  to  

[8, 5, 1]
[7, 3, 2]
[4, 0, 6]
  to  

[8, 5, 1]
[7, 3, 2]
[0, 4, 6]
  to  

[8, 5, 1]
[0, 3, 2]
[7, 4, 6]
  to  

[8, 5, 1]
[3, 0, 2]
[7, 4, 6]
  to  

[8, 0, 1]
[3, 5, 2]
[7, 4, 6]
  to  

[0, 8, 1]
[3, 5, 2]
[7, 4, 6]
  to  

[3, 8, 1]
[0, 5, 2]
[7, 4, 6]
  to  

[3, 8, 1]
[5, 0, 2]
[7, 4, 6]
  to  

[3, 0, 1]
[5, 8, 2]
[7, 4, 6]
  to  

[3, 1, 0]
[5, 8, 2]
[7, 4, 6]
  to  

[3, 1, 2]
[5, 8, 0]
[7, 4, 6]
  to  

[3, 1, 2]
[5, 0, 8]
[7, 4, 6]
  to  

[3, 1, 2]
[0, 5, 8]
[7, 4, 6]
  to  

[3, 1, 2]
[7, 5, 8]
[0, 4, 6]
  to  

[3, 1, 2]
[7, 5, 8]
[4, 0, 6]
  to  

[3, 1, 2]
[7, 5, 8]
[4, 6, 0]
  to  

[3, 1, 2]
[7, 5, 0]
[4, 6, 8]
  to  

[3, 1, 2]
[7, 0, 5]
[4, 6, 8]
  to  

[3, 1, 2]
[0, 7, 5]
[4, 6, 8]
  to  

[3, 1, 2]
[4, 7, 5]
[0, 6, 8]
  to  

[3, 1, 2]
[4, 7, 5]
[6, 0, 8]
  to  

[3, 1, 2]
[4, 0, 5]
[6, 7, 8]
  to  

[3, 1, 2]
[0, 4, 5]
[6, 7, 8]
  to  

  

[0, 1,


Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
22 nodes traced.

[6, 5, 8]
[0, 2, 1]
[7, 4, 3]
  to  

[6, 5, 8]
[2, 0, 1]
[7, 4, 3]
  to  

[6, 5, 8]
[2, 1, 0]
[7, 4, 3]
  to  

[6, 5, 0]
[2, 1, 8]
[7, 4, 3]
  to  

[6, 0, 5]
[2, 1, 8]
[7, 4, 3]
  to  

[6, 1, 5]
[2, 0, 8]
[7, 4, 3]
  to  

[6, 1, 5]
[0, 2, 8]
[7, 4, 3]
  to  

[0, 1, 5]
[6, 2, 8]
[7, 4, 3]
  to  

[1, 0, 5]
[6, 2, 8]
[7, 4, 3]
  to  

[1, 2, 5]
[6, 0, 8]
[7, 4, 3]
  to  

[1, 2, 5]
[6, 4, 8]
[7, 0, 3]
  to  

[1, 2, 5]
[6, 4, 8]
[7, 3, 0]
  to  

[1, 2, 5]
[6, 4, 0]
[7, 3, 8]
  to  

[1, 2, 5]
[6, 0, 4]
[7, 3, 8]
  to  

[1, 2, 5]
[6, 3, 4]
[7, 0, 8]
  to  

[1, 2, 5]
[6, 3, 4]
[0, 7, 8]
  to  

[1, 2, 5]
[0, 3, 4]
[6, 7, 8]
  to  

[1, 2, 5]
[3, 0, 4]
[6, 7, 8]
  to  

[1, 2, 5]
[3, 4, 0]
[6, 7, 8]
  to  

[1, 2, 0]
[3, 4, 5]
[6, 7, 8]
  to  

[1, 0, 2]
[3, 4, 5]
[6, 7, 8]
  to  

  

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
To solve this problem the search algorithm expanded a total of 4785 nodes.
The maximum num

...


Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
24 nodes traced.

[8, 2, 7]
[0, 6, 5]
[4, 1, 3]
  to  

[8, 2, 7]
[4, 6, 5]
[0, 1, 3]
  to  

[8, 2, 7]
[4, 6, 5]
[1, 0, 3]
  to  

[8, 2, 7]
[4, 6, 5]
[1, 3, 0]
  to  

[8, 2, 7]
[4, 6, 0]
[1, 3, 5]
  to  

[8, 2, 0]
[4, 6, 7]
[1, 3, 5]
  to  

[8, 0, 2]
[4, 6, 7]
[1, 3, 5]
  to  

[0, 8, 2]
[4, 6, 7]
[1, 3, 5]
  to  

[4, 8, 2]
[0, 6, 7]
[1, 3, 5]
  to  

[4, 8, 2]
[1, 6, 7]
[0, 3, 5]
  to  

[4, 8, 2]
[1, 6, 7]
[3, 0, 5]
  to  

[4, 8, 2]
[1, 0, 7]
[3, 6, 5]
  to  

[4, 0, 2]
[1, 8, 7]
[3, 6, 5]
  to  

[0, 4, 2]
[1, 8, 7]
[3, 6, 5]
  to  

[1, 4, 2]
[0, 8, 7]
[3, 6, 5]
  to  

[1, 4, 2]
[3, 8, 7]
[0, 6, 5]
  to  

[1, 4, 2]
[3, 8, 7]
[6, 0, 5]
  to  

[1, 4, 2]
[3, 0, 7]
[6, 8, 5]
  to  

[1, 4, 2]
[3, 7, 0]
[6, 8, 5]
  to  

[1, 4, 2]
[3, 7, 5]
[6, 8, 0]
  to  

[1, 4, 2]
[3, 7, 5]
[6, 0, 8]
  to  

[1, 4, 2]
[3, 0, 5]
[6, 7, 8]
  to  

[1, 0, 2]
[3, 4, 5]
[6, 7, 8]
  to  

  

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
To solve this 


Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
25 nodes traced.

[8, 2, 6]
[1, 0, 5]
[7, 3, 4]
  to  

[8, 2, 6]
[1, 3, 5]
[7, 0, 4]
  to  

[8, 2, 6]
[1, 3, 5]
[7, 4, 0]
  to  

[8, 2, 6]
[1, 3, 0]
[7, 4, 5]
  to  

[8, 2, 0]
[1, 3, 6]
[7, 4, 5]
  to  

[8, 0, 2]
[1, 3, 6]
[7, 4, 5]
  to  

[0, 8, 2]
[1, 3, 6]
[7, 4, 5]
  to  

[1, 8, 2]
[0, 3, 6]
[7, 4, 5]
  to  

[1, 8, 2]
[3, 0, 6]
[7, 4, 5]
  to  

[1, 0, 2]
[3, 8, 6]
[7, 4, 5]
  to  

[0, 1, 2]
[3, 8, 6]
[7, 4, 5]
  to  

[3, 1, 2]
[0, 8, 6]
[7, 4, 5]
  to  

[3, 1, 2]
[7, 8, 6]
[0, 4, 5]
  to  

[3, 1, 2]
[7, 8, 6]
[4, 0, 5]
  to  

[3, 1, 2]
[7, 0, 6]
[4, 8, 5]
  to  

[3, 1, 2]
[7, 6, 0]
[4, 8, 5]
  to  

[3, 1, 2]
[7, 6, 5]
[4, 8, 0]
  to  

[3, 1, 2]
[7, 6, 5]
[4, 0, 8]
  to  

[3, 1, 2]
[7, 0, 5]
[4, 6, 8]
  to  

[3, 1, 2]
[0, 7, 5]
[4, 6, 8]
  to  

[3, 1, 2]
[4, 7, 5]
[0, 6, 8]
  to  

[3, 1, 2]
[4, 7, 5]
[6, 0, 8]
  to  

[3, 1, 2]
[4, 0, 5]
[6, 7, 8]
  to  

[3, 1, 2]
[0, 4, 5]
[6, 7, 8]
  to  

  

[0, 1,


Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
26 nodes traced.

[2, 1, 3]
[0, 7, 8]
[6, 5, 4]
  to  

[2, 1, 3]
[6, 7, 8]
[0, 5, 4]
  to  

[2, 1, 3]
[6, 7, 8]
[5, 0, 4]
  to  

[2, 1, 3]
[6, 0, 8]
[5, 7, 4]
  to  

[2, 0, 3]
[6, 1, 8]
[5, 7, 4]
  to  

[0, 2, 3]
[6, 1, 8]
[5, 7, 4]
  to  

[6, 2, 3]
[0, 1, 8]
[5, 7, 4]
  to  

[6, 2, 3]
[5, 1, 8]
[0, 7, 4]
  to  

[6, 2, 3]
[5, 1, 8]
[7, 0, 4]
  to  

[6, 2, 3]
[5, 1, 8]
[7, 4, 0]
  to  

[6, 2, 3]
[5, 1, 0]
[7, 4, 8]
  to  

[6, 2, 3]
[5, 0, 1]
[7, 4, 8]
  to  

[6, 0, 3]
[5, 2, 1]
[7, 4, 8]
  to  

[6, 3, 0]
[5, 2, 1]
[7, 4, 8]
  to  

[6, 3, 1]
[5, 2, 0]
[7, 4, 8]
  to  

[6, 3, 1]
[5, 0, 2]
[7, 4, 8]
  to  

[6, 3, 1]
[0, 5, 2]
[7, 4, 8]
  to  

[0, 3, 1]
[6, 5, 2]
[7, 4, 8]
  to  

[3, 0, 1]
[6, 5, 2]
[7, 4, 8]
  to  

[3, 1, 0]
[6, 5, 2]
[7, 4, 8]
  to  

[3, 1, 2]
[6, 5, 0]
[7, 4, 8]
  to  

[3, 1, 2]
[6, 0, 5]
[7, 4, 8]
  to  

[3, 1, 2]
[6, 4, 5]
[7, 0, 8]
  to  

[3, 1, 2]
[6, 4, 5]
[0, 7, 8]
  to  

[3, 1, 2]



Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
19 nodes traced.

[7, 4, 1]
[5, 3, 2]
[6, 8, 0]
  to  

[7, 4, 1]
[5, 3, 2]
[6, 0, 8]
  to  

[7, 4, 1]
[5, 3, 2]
[0, 6, 8]
  to  

[7, 4, 1]
[0, 3, 2]
[5, 6, 8]
  to  

[0, 4, 1]
[7, 3, 2]
[5, 6, 8]
  to  

[4, 0, 1]
[7, 3, 2]
[5, 6, 8]
  to  

[4, 3, 1]
[7, 0, 2]
[5, 6, 8]
  to  

[4, 3, 1]
[0, 7, 2]
[5, 6, 8]
  to  

[4, 3, 1]
[5, 7, 2]
[0, 6, 8]
  to  

[4, 3, 1]
[5, 7, 2]
[6, 0, 8]
  to  

[4, 3, 1]
[5, 0, 2]
[6, 7, 8]
  to  

[4, 3, 1]
[0, 5, 2]
[6, 7, 8]
  to  

[0, 3, 1]
[4, 5, 2]
[6, 7, 8]
  to  

[3, 0, 1]
[4, 5, 2]
[6, 7, 8]
  to  

[3, 1, 0]
[4, 5, 2]
[6, 7, 8]
  to  

[3, 1, 2]
[4, 5, 0]
[6, 7, 8]
  to  

[3, 1, 2]
[4, 0, 5]
[6, 7, 8]
  to  

[3, 1, 2]
[0, 4, 5]
[6, 7, 8]
  to  

  

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
To solve this problem the search algorithm expanded a total of 1261 nodes.
The maximum number of nodes in the queue at any one time: 798
--- 0.3261855000000651 seconds For misplaced ---
##################


Found a solution!
Traceback:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
25 nodes traced.

[7, 5, 8]
[3, 4, 1]
[0, 6, 2]
  to  

[7, 5, 8]
[3, 4, 1]
[6, 0, 2]
  to  

[7, 5, 8]
[3, 4, 1]
[6, 2, 0]
  to  

[7, 5, 8]
[3, 4, 0]
[6, 2, 1]
  to  

[7, 5, 0]
[3, 4, 8]
[6, 2, 1]
  to  

[7, 0, 5]
[3, 4, 8]
[6, 2, 1]
  to  

[0, 7, 5]
[3, 4, 8]
[6, 2, 1]
  to  

[3, 7, 5]
[0, 4, 8]
[6, 2, 1]
  to  

[3, 7, 5]
[4, 0, 8]
[6, 2, 1]
  to  

[3, 7, 5]
[4, 8, 0]
[6, 2, 1]
  to  

[3, 7, 5]
[4, 8, 1]
[6, 2, 0]
  to  

[3, 7, 5]
[4, 8, 1]
[6, 0, 2]
  to  

[3, 7, 5]
[4, 0, 1]
[6, 8, 2]
  to  

[3, 0, 5]
[4, 7, 1]
[6, 8, 2]
  to  

[3, 5, 0]
[4, 7, 1]
[6, 8, 2]
  to  

[3, 5, 1]
[4, 7, 0]
[6, 8, 2]
  to  

[3, 5, 1]
[4, 7, 2]
[6, 8, 0]
  to  

[3, 5, 1]
[4, 7, 2]
[6, 0, 8]
  to  

[3, 5, 1]
[4, 0, 2]
[6, 7, 8]
  to  

[3, 0, 1]
[4, 5, 2]
[6, 7, 8]
  to  

[3, 1, 0]
[4, 5, 2]
[6, 7, 8]
  to  

[3, 1, 2]
[4, 5, 0]
[6, 7, 8]
  to  

[3, 1, 2]
[4, 0, 5]
[6, 7, 8]
  to  

[3, 1, 2]
[0, 4, 5]
[6, 7, 8]
  to  

  

[0, 1,


# The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Solution of the first Scenario:
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### .
#### .
#### .
#### 0 1 2
#### 3 4 5
#### 6 7 8

                    Average_Steps    Average_Time      
 Miss Placed
 
 Manhattan Distance

